In [2]:
import numpy as np
from osgeo import gdal
from scipy.ndimage import distance_transform_edt
import os

# (Keep the align_raster_to_reference function as it was)
def align_raster_to_reference(source_path, reference_ds, resampling_method='near'):
    # ... (Function contents unchanged) ...
    ref_gt = reference_ds.GetGeoTransform()
    ref_proj = reference_ds.GetProjection()
    ref_xsize = reference_ds.RasterXSize
    ref_ysize = reference_ds.RasterYSize
    
    warp_options = gdal.WarpOptions(
        format='MEM',
        resampleAlg=resampling_method,
        targetAlignedPixels=True,
        outputBounds=(ref_gt[0], ref_gt[3] + ref_ysize * ref_gt[5], ref_gt[0] + ref_xsize * ref_gt[1], ref_gt[3]),
        xRes=ref_gt[1],
        yRes=ref_gt[5],
        dstSRS=ref_proj,
        multithread=True
    )

    aligned_ds = gdal.Warp('', source_path, options=warp_options)
    return aligned_ds

# ----------------------------------------------------------------------

def create_scaled_r1_priority_mosaic(raster_path_low_priority, raster_path_high_priority, output_path, r1_scale_factor=1.0):
    """
    Creates a mosaic where R1 (scaled) overwrites R2, prioritizing R1 wherever it has non-zero data.
    """
    
    print("--- Starting Alignment and Scaled R1 Priority Process ---")

    # --- 1. Load Rasters and Align ---
    ds_r2_ref = gdal.Open(raster_path_high_priority, gdal.GA_ReadOnly)
    if not ds_r2_ref:
        print("Error: Could not open R2 (High Priority).")
        return

    print(f"Aligning R1 to R2's grid...")
    ds_r1_aligned = align_raster_to_reference(raster_path_low_priority, ds_r2_ref, resampling_method='bilinear')
    
    band_r1 = ds_r1_aligned.GetRasterBand(1)
    band_r2 = ds_r2_ref.GetRasterBand(1)
    R1_data_full = band_r1.ReadAsArray().astype(np.float32)
    R2_data_full = band_r2.ReadAsArray().astype(np.float32)

    # --- FIX: Synchronize Array Shapes ---
    shape_r1 = R1_data_full.shape
    shape_r2 = R2_data_full.shape
    min_rows = min(shape_r1[0], shape_r2[0])
    min_cols = min(shape_r1[1], shape_r2[1])

    R1_data = R1_data_full[:min_rows, :min_cols]
    R2_data = R2_data_full[:min_rows, :min_cols]
    print(f"Rasters synced to common shape: {R1_data.shape}")

    # --- 2. Create Masks and Overlap ---
    nodata_r1 = band_r1.GetNoDataValue() if band_r1.GetNoDataValue() is not None else -9999.0
    nodata_r2 = band_r2.GetNoDataValue() if band_r2.GetNoDataValue() is not None else -9999.0
    
    R1_data[np.isnan(R1_data)] = nodata_r1
    R2_data[np.isnan(R2_data)] = nodata_r2

    # Apply scaling factor to R1 data only to the actual data values
    valid_r1_idx = (R1_data != nodata_r1)
    R1_data[valid_r1_idx] = R1_data[valid_r1_idx] * r1_scale_factor
    print(f"R1 values scaled by factor: {r1_scale_factor}")

    R1_mask = (R1_data != nodata_r1).astype(np.uint8)
    R2_mask = (R2_data != nodata_r2).astype(np.uint8)
    
    # --- 3. Apply R1 Priority Overwrite ---

    # 3a. Initialize the final output array with R2's data (the full domain)
    # R2 is the lower priority base across the entire extent
    final_data = np.copy(R2_data)
    
    # Identify all indices where R1 has valid, non-zero data
    R1_non_zero_idx = np.where((R1_mask == 1) & (R1_data != 0))

    # 3b. Overwrite the final array with R1 data wherever R1 is valid and non-zero
    # This step ensures R1 overwrites R2 completely in the overlap area.
    final_data[R1_non_zero_idx] = R1_data[R1_non_zero_idx]

    # 3c. Handle final NoData values
    # Areas where R1 was non-data AND R2 was non-data (or where R2 was used as base)
    # We don't need to re-apply the R2-only logic as R2 is the base array.
    # The NoData in R2's non-data areas remains nodata_r2.
    final_data[(R1_mask == 0) & (R2_mask == 0)] = nodata_r2
    
    # --- 4. Write Output Raster (STABLE VERSION) ---
    
    gt_r2 = ds_r2_ref.GetGeoTransform()
    
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(
        output_path, 
        min_cols, 
        min_rows, 
        1, 
        band_r2.DataType, 
        options = ['COMPRESS=LZW', 'TILED=YES']
    )
    
    out_ds.SetProjection(ds_r2_ref.GetProjection())
    out_ds.SetGeoTransform(gt_r2)

    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(final_data)
    out_band.SetNoDataValue(nodata_r2)
    out_band.FlushCache()
    
    del out_ds, ds_r2_ref, ds_r1_aligned 
    
    print(f"✅ Scaled R1 priority mosaic successfully saved to: {output_path}")

# --- Example Usage ---
# create_scaled_r1_priority_mosaic(R1_LOW_PRIORITY, R2_HIGH_PRIORITY, OUTPUT_MOSAIC, r1_scale_factor=0.9)

In [5]:
R1_LOW_PRIORITY = "E:/GOES-R Lightning Data/EAST-MEAN-FINAL/east_mean_energy_2025_wgs84.tif"
R2_HIGH_PRIORITY = "E:/GOES-R Lightning Data/WEST-MEAN-FINAL/west_mean_energy_2025_wgs84.tif"
OUTPUT_MOSAIC = "E:/GOES-R Lightning Data/JOINED-MEAN-FINAL/mean_energy_2025_wgs84.tif"

create_scaled_r1_priority_mosaic(R1_LOW_PRIORITY, R2_HIGH_PRIORITY, OUTPUT_MOSAIC, 0.65)

--- Starting Alignment and Scaled R1 Priority Process ---
Aligning R1 to R2's grid...
Rasters synced to common shape: (2997, 7061)
R1 values scaled by factor: 0.65
✅ Scaled R1 priority mosaic successfully saved to: E:/GOES-R Lightning Data/JOINED-MEAN-FINAL/mean_energy_2025_wgs84.tif
